In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import argparse
from silver_control import get_last_processed_run_id, update_last_processed_run_id, get_new_run_ids, get_all_run_ids


def load_fact_procedures_incremental(spark: SparkSession, catalog_name: str, run_ids: list[str]):
    """Load fact_procedures table incrementally."""
    
    run_ids_str = "', '".join(run_ids)
    
    df = spark.sql(f"""
        SELECT
            md5(concat(pr.PATIENT, pr.ENCOUNTER, cast(pr.CODE as string), cast(pr.START as string))) as procedure_id,
            pr.PATIENT as patient_id,
            pr.ENCOUNTER as encounter_id,
            pr.START as procedure_start_timestamp,
            pr.STOP as procedure_stop_timestamp,
            DATE(pr.START) as procedure_date_key,
            (unix_timestamp(pr.STOP) - unix_timestamp(pr.START)) / 60 as procedure_duration_minutes,
            pr.CODE as procedure_code,
            pr.DESCRIPTION as procedure_description,
            pr.SYSTEM as code_system,
            pr.BASE_COST as procedure_cost,
            pr.REASONCODE as reason_code,
            pr.REASONDESCRIPTION as reason_description,
            1 as procedure_count,
            p.GENDER as patient_gender,
            p.BIRTHDATE as patient_birthdate,
            year(pr.START) - year(p.BIRTHDATE) as patient_age_at_procedure,
            p.STATE as patient_state,
            p.ZIP as patient_zip,
            e.ENCOUNTERCLASS as encounter_class,
            e.PROVIDER as provider_id,
            e.ORGANIZATION as organization_id,
            e.PAYER as payer_id,
            prov.SPECIALITY as provider_specialty,
            prov.NAME as provider_name,
            org.NAME as organization_name,
            org.CITY as organization_city,
            org.STATE as organization_state,
            pr.ingest_run_id,
            pr.ingest_timestamp,
            current_timestamp() as silver_load_timestamp
        FROM {catalog_name}.synthea.procedures_bronze pr
        LEFT JOIN {catalog_name}.synthea.patients_bronze p 
            ON pr.PATIENT = p.Id
        LEFT JOIN {catalog_name}.synthea.encounters_bronze e 
            ON pr.ENCOUNTER = e.Id
        LEFT JOIN {catalog_name}.synthea.providers_bronze prov 
            ON e.PROVIDER = prov.Id
        LEFT JOIN {catalog_name}.synthea.organizations_bronze org 
            ON e.ORGANIZATION = org.Id
        WHERE pr.ingest_run_id IN ('{run_ids_str}')
    """)
    
    return df


def main():
    spark = SparkSession.getActiveSession() or SparkSession.builder.getOrCreate()
    # Get parameters from Databricks widgets (for notebook tasks)
    try:
        from pyspark.dbutils import DBUtils
        dbutils = DBUtils(spark)
        catalog_name = dbutils.widgets.get("catalog_name")
        load_type = dbutils.widgets.get("load_type") if dbutils.widgets.get("load_type") else "full"
    except Exception:
        # Fallback to argparse for Python script tasks
        parser = argparse.ArgumentParser()
        parser.add_argument("--catalog_name", type=str, required=True)
        parser.add_argument("--load_type", type=str, default="full", choices=["full", "incremental"])
        args, _ = parser.parse_known_args()
        catalog_name = args.catalog_name
        load_type = args.load_type
    table_name = "procedures_silver"
    
    # Get run IDs to process based on load type
    if load_type == "full":
        print("Running FULL load - processing all data")
        run_ids = get_all_run_ids(spark, catalog_name, "procedures_bronze")
    else:
        # Incremental load
        last_run_id = get_last_processed_run_id(spark, catalog_name, table_name)
        print(f"Running INCREMENTAL load - Last processed run ID: {last_run_id}")
        run_ids = get_new_run_ids(spark, catalog_name, "procedures_bronze", last_run_id)
    
    if not run_ids:
        print("No data to process.")
        return
    
    print(f"Processing {len(run_ids)} run(s): {run_ids}")
    
    df = load_fact_procedures_incremental(spark, catalog_name, run_ids)
    
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.synthea")
    
    # Write to silver table - use overwrite for full loads, append for incremental
    write_mode = "overwrite" if load_type == "full" else "append"
    (df.write
        .mode(write_mode)
        .format("delta")
        .option("mergeSchema", "true")
        .saveAsTable(f"{catalog_name}.synthea.{table_name}"))
    
    update_last_processed_run_id(spark, catalog_name, table_name, run_ids[-1])
    
    record_count = df.count()
    print(f"✓ Loaded {record_count} records into {table_name} (mode: {load_type})")


if __name__ == "__main__":
    main()

